In [1]:
import numpy as np
from math import log, sqrt, exp
from scipy import stats

### European call by Monte Carlo

- initial stock level $S_0$ = 100
- strike of the call $K$ = 105
- Time to maturity $T$ = 1 year
- risk-free rate $r$ = 0.05
- volatility $\sigma$ = 0.2

stock level at maturity in BS model:
$$S_T = S_0 exp\left( \left( r - \frac{1}{2}\sigma^2\right)T + \sigma\sqrt{T}z \right)$$
where $z$ is a standard normal random variable.

- The non-arbitrage value of european call is given by  the expectation of the discounted cash flows.  
- Here one cash flow at maturity $T$: $max(S_T - K, 0)$
flat rate $r$, therefore discount factor for maturity $T$ : $e^{-rT}$
- The non-arbitrage value is then: 

$$e^{-rT}\mathbb{E}\left[max(S_T - K, 0)\right]$$

To value the call by **Monte Carlo**:
1. Draw $N$ random numbers from the standard normal distribution: $z_1,...,z_N$
2. Calculate all the stock levels at time $T$: $S_T^1,...,S_T^N$ using the $z_i$ numbers.
3. Calculate the payoff values at maturity: $G_T^i = max(S_T^i - K, 0)$ that is $G_T^1,..., G_T^N$.
4. Finally, estimate the present value $V_0$ by forming the average of the simulated payoff values. And discount from $T$ to time $0$:
$$V_0 \approx e^{-rT} \frac{1}{N}\sum_{i=1}^N G_T^i$$

exemple SC model : $S(T) \sim N(S_0, \sigma)$  
where $S_0 = 100$ and $\sigma = 20$  
assume $r=0$, so $df = 1$

In [10]:
def payoff_func(x):
    return np.maximum(x - K, 0)

In [28]:
N = 10000 # sample size
S0 = 100
stddev = 20
K = 102
# np.random.normal(mean=..., stddev=..., size=None)
Ss = np.random.normal(S0, stddev, size=N) # draw N numbers from S(T)
Gs = payoff_func(Ss)
mc_mean = Gs.mean()
print(f"{N}-sample mean is: {mc_mean}")

10000-sample mean is: 6.9543926815792565


In [33]:
def mc_call_pv(S0,K,T,r,sigma,num_paths):
    Z = np.random.normal(size=num_paths) # N samples of N(0,1)
    S_T = S0 * np.exp( (r - 0.5*sigma**2)*T + sigma*np.sqrt(T)*Z ) # N samples of S(T)
    G_T = payoff_func(S_T) # N samples of G(T)
    mc_pv = G_T.mean() * np.exp(-r*T)
    return mc_pv

In [34]:
#  parameter values
S0 = 100
K = 105
T = 1
r = 0.05
sigma = 0.2
n1 = 100
n2 = 1000
n3 = 10000

print(f"{n1}-sample mean is: {mc_call_pv(S0,K,T,r,sigma,n1)}")
print(f"{n1}-sample mean is: {mc_call_pv(S0,K,T,r,sigma,n1)}")
print(f"{n1}-sample mean is: {mc_call_pv(S0,K,T,r,sigma,n1)}")
print(f"{n1}-sample mean is: {mc_call_pv(S0,K,T,r,sigma,n1)} \n")

print(f"{n2}-sample mean is: {mc_call_pv(S0,K,T,r,sigma,n2)}")
print(f"{n2}-sample mean is: {mc_call_pv(S0,K,T,r,sigma,n2)}")
print(f"{n2}-sample mean is: {mc_call_pv(S0,K,T,r,sigma,n2)}")
print(f"{n2}-sample mean is: {mc_call_pv(S0,K,T,r,sigma,n2)} \n")

print(f"{n3}-sample mean is: {mc_call_pv(S0,K,T,r,sigma,n3)}")
print(f"{n3}-sample mean is: {mc_call_pv(S0,K,T,r,sigma,n3)}")
print(f"{n3}-sample mean is: {mc_call_pv(S0,K,T,r,sigma,n3)}")
print(f"{n3}-sample mean is: {mc_call_pv(S0,K,T,r,sigma,n3)}")

100-sample mean is: 8.70201351666766
100-sample mean is: 7.559389827695227
100-sample mean is: 7.947533366179867
100-sample mean is: 7.943956019033248 

1000-sample mean is: 7.8449787789971115
1000-sample mean is: 8.162594740198797
1000-sample mean is: 7.999837158260249
1000-sample mean is: 7.606180736439126 

10000-sample mean is: 7.799715237133487
10000-sample mean is: 7.937878602208611
10000-sample mean is: 8.150092177059184
10000-sample mean is: 8.075672554532632


### European call by B-S formula

The value at time $t$ of a European call for in [B-S formula](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model#Black%E2%80%93Scholes_formula) with the same parameters as above is:

\begin{equation*}
\begin{aligned}
 V_t &= N(d_1) S_t - N(d_2)Ke^{-rT} \\
 d_1 &= \frac{ln \left( \frac{S_t}{K} \right) + \left( r + \frac{\sigma^2}{2} \right)(T-t)}{\sigma \sqrt{T-t}}  \\
 d_2 &= d_1 - \sigma \sqrt{T-t}
\end{aligned}
\end{equation*}

The price of a corresponding put option of same strike $K$ and maturiry $T$ is:

$$V_t = N(-d_2)Ke^{-rT} - N(-d_1)S_t$$

where $N(.)$ is the cumulative distribution function of the standard normal distribution. $(T-t)$ is the time to matutity (expressed in years). 

Note that for the time $t=0$ call value, we simply write:

\begin{equation*}
\begin{aligned}
 V_0 &= N(d_1) S_0 - N(d_2)Ke^{-rT} \\
 d_1 &= \frac{ln \left( \frac{S_0}{K} \right) + \left( r + \frac{\sigma^2}{2} \right)T}{\sigma \sqrt{T}}  \\
 d_2 &= d_1 - \sigma \sqrt{T}
\end{aligned}
\end{equation*}